[Reference](https://medium.com/@Rohan_Dutt/why-feature-stores-are-essential-for-consistency-in-ml-and-how-to-build-a-simple-one-969cc38599a9)

# Core Architecture

In [1]:
from dataclasses import dataclass

# The Single Source of Truth
# I define the logic ONCE here. No copy-pasting into SQL files.
@dataclass
class FeatureDefinition:
    name: str
    entity_id: str
    description: str
    # The logic that runs in the Offline Store (Batch)
    batch_sql: str = """
        SELECT user_id, count(*) as val
        FROM logs
        WHERE action = 'click'
        AND timestamp > current_date - interval '7' day
        GROUP BY user_id
    """
    # The storage type for the Online Store (Inference)
    online_type: str = "int32"
click_feature = FeatureDefinition(
    name="user_clicks_7d",
    entity_id="user_id",
    description="Total clicks in last 7 days"
)


In [2]:
import pandas as pd
import redis

# The "Materialization" Job
# This runs every hour to keep the Online Store fresh.
def sync_offline_to_online(feature_def, redis_client):
    # 1. Get the heavy data from the Offline Warehouse
    df = pd.read_sql(feature_def.batch_sql, offline_db_connection)

    # 2. Push it to the Online Cache (Fast access)
    # Using a pipeline makes this blazingly fast.
    pipe = redis_client.pipeline()
    for _, row in df.iterrows():
        # Key: "user_id:123", Field: "user_clicks_7d", Value: 50
        key = f"{feature_def.entity_id}:{row[feature_def.entity_id]}"
        pipe.hset(key, feature_def.name, row['val'])

    pipe.execute()
    print(f"Synced {len(df)} records for {feature_def.name}.")
# usage
# sync_offline_to_online(click_feature, r)

# Building a “DIY” Feature Store

In [3]:
import redis
import json

# My Simple SDK
def get_online_features(entity_ids, feature_names, redis_client):
    # Pipeline is the secret sauce.
    # It sends ONE request instead of hundreds.
    pipe = redis_client.pipeline()

    for entity_id in entity_ids:
        # Key format: "user:12345"
        pipe.hmget(f"user:{entity_id}", feature_names)

    results = pipe.execute()

    # Format the mess into clean JSON
    features = []
    for i, entity_id in enumerate(entity_ids):
        features.append({
            "entity_id": entity_id,
            "values": dict(zip(feature_names, results[i]))
        })

    return features

In [4]:
# The Time Machine (PySpark)
# I save this to S3 for the offline store
df = spark.read.parquet("s3://raw-logs/")

# Calculate features for a specific day in the past
daily_features = df.filter(df.date == "2023-10-27") \
    .groupBy("user_id") \
    .agg({"amount": "sum"}) \
    .withColumnRenamed("sum(amount)", "total_spend_1d")
# Write to partitioned folders
daily_features.write.partitionBy("date").parquet("s3://feature-store/offline/")

In [5]:
# The Safety Net
# When I sync data, I make sure it dies eventually.
def sync_to_redis(user_id, features, redis_client):
    key = f"user:{user_id}"

    # Set the data
    redis_client.hset(key, mapping=features)

    # Expire after 7 days (604800 seconds)
    # If the user stops visiting, we stop paying for storage.
    redis_client.expire(key, 604800)

# Advanced Patterns for Scale

In [6]:
# On-Demand Logic
# The Feature Store fetches 'home_loc' from Redis (Fast)
# But it calculates 'dist' using the Request Context (Live)

def get_distance_feature(user_id, current_lat, current_lon, redis_client):
    # 1. Get static data
    home_loc = json.loads(redis_client.get(f"user:{user_id}:home"))

    # 2. Compute dynamic feature NOW
    # If I pre-computed this, it would be wrong by the time you read it.
    return haversine(current_lat, current_lon, home_loc['lat'], home_loc['lon'])

In [7]:
from scipy.stats import entropy
import numpy as np

# The Panic Button 🚨
# I run this check every hour.
def check_drift(training_dist, live_dist):
    # KL Divergence: 0 means identical. High means trouble.
    score = entropy(training_dist, live_dist)

    if score > 0.1:
        raise Alert(f"Data is drifting! Score: {score}")

    return "All good."